In [30]:
from datasets import load_dataset, Dataset, concatenate_datasets

dataset = load_dataset("blinoff/medical_qa_ru_data")
dataset = dataset["train"]
dataset

Found cached dataset medical_qa_ru_data (/home/kosenko/.cache/huggingface/datasets/blinoff___medical_qa_ru_data/default/0.0.0/551476392fb5023cebc55b480ef3948c0ad2287c0ef0ad158a56fbb0bc37783f)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['date', 'categ', 'theme', 'desc', 'ans', 'spec10'],
    num_rows: 190335
})

In [ ]:
# dataset.push_to_hub("dim/medical_qa_ru_data", private=True)

In [31]:
dataset = dataset.filter(lambda x: x["spec10"].strip() != "")

Loading cached processed dataset at /home/kosenko/.cache/huggingface/datasets/blinoff___medical_qa_ru_data/default/0.0.0/551476392fb5023cebc55b480ef3948c0ad2287c0ef0ad158a56fbb0bc37783f/cache-a0a1b65e4d41d507.arrow


In [10]:
dataset[0]

{'date': '8 Октября 2017, 11:55',
 'categ': 'Оториноларингология',
 'theme': 'Применение Ларипронта.',
 'desc': 'Ларипронт 20 талеток,через каждые 2-3 часа.Очень понравились,но пока принимала,чувствовала себя хорошо. Закончились-все сиптомы опять проявились. Нигде не найду как долго их можно принимать. Скажите пожалуйста.',
 'ans': 'Что вы им лечите? Длительность приема Ларипронта индивидуальна и устанавливается лечащим врачом.;\nУточните у своего лечащего врача.',
 'spec10': 'Отоларинголог'}

In [36]:
import numpy as np

answer_lens = []
for ans in dataset["ans"]:
    ans = ans.split(";\n")[0].split()
    answer_lens.append(len(ans))

In [39]:
np.percentile(np.array(answer_lens), 80)

16.0

In [41]:
prompts = []

for i, item in enumerate(dataset):
    theme = item["theme"]
    desc = item["desc"]
    ans = item["ans"]
    prompt = f"{theme}\n{desc}"
    content = ans.split(";\n")[0]
    if len(content.split()) > 3:
        prompts.append(
            {
                "prompt": prompt,
                "content": content,
            }
        )
    # print(prompt)
    # print("="*10)
    # print(ans.split(";\n")[0])
    # print("="*100)
    # if i > 5000:
    #     break

In [43]:
len(prompts)

80101

In [44]:
print(prompts[102]["prompt"])
print("=" * 10)
print(prompts[103]["content"].split(";\n")[0])

мне 56 лет это опасное изменении предстатнльной железы
Длина 5,2 см
            
            Толщина 4,8 см
            
            Ширина 4,9 см обьем 63,60 см кубические
            
            в приуретной зоне  лоцируется эхо(+) включая  в Д 0,4 см и 0,5 см с акустической тенью
            
            мочевой пузырь  560 см кубических
            
            после опорожения 126 см кубических
Нужно сдать ОАК, коагулограмму. Консультация гематолога.


In [45]:
prompts = Dataset.from_list(prompts)

In [46]:
prompts[210]

{'prompt': '.\nЗдравствуйте, хочу попробовать Элтацин, как думаете он поможет от ВСД или нет?',
 'content': 'У препарата четкие показания есть,если они у вас присутствуют,то принимайте.'}

In [47]:
prompts

Dataset({
    features: ['prompt', 'content'],
    num_rows: 80101
})

In [48]:
prompts.push_to_hub("dim/medical_qa_ru_prompts")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/81 [00:00<?, ?ba/s]

In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets

dataset = load_dataset("dim/medical_qa_ru_prompts")
dataset = dataset["train"]
dataset

In [7]:
dataset[16]

{'prompt': 'Профилактика простуды при беременности\nЗдравствуйте, муж заболел у него насморк и температура, что можно мне пропить чтобы не заболеть? Я беременна на 33 неделе',
 'content': 'Носите маску,соблюдайте правила личной гигиены.'}